In [0]:
%tensorflow_version 2.x
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from nltk import word_tokenize

import nltk
nltk.download('punkt')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout, Activation,Embedding

from sklearn.metrics import roc_auc_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
spam_data = pd.read_pickle(r'/content/drive/My Drive/Colab Notebooks/spam_data_df1.pkl')

In [4]:
spam_data.head()

,question_clean,target
0,quebec nationalist see provinc nation,0
1,adopt dog would encourag peopl adopt shop,0
2,veloc affect time veloc affect space geometry,0
3,otto von guerick use magdeburg hemispheres,0
4,convert montra helicon mountain bike chang tyres,0


In [0]:
train,test = train_test_split(spam_data,test_size=0.3,stratify= spam_data.target,random_state=2)

In [0]:
x_train= train.question_clean
y_train= train.target

x_test=test.question_clean
y_test=test.target

In [7]:
x_train.shape , y_train.shape, x_test.shape, y_test.shape

((914285,), (914285,), (391837,), (391837,))

In [0]:
question_len=[]

for quest in x_train:
    question_len.append(len(word_tokenize(quest)))


In [9]:
max(question_len)

67

In [10]:
np.quantile(question_len,0.9999)

29.571600000024773

In [0]:
question_len1=[]

for quest in x_test:
    question_len1.append(len(word_tokenize(quest)))

In [12]:
print(max(question_len1))

np.quantile(question_len1,0.9999)

36


29.0

In [0]:
max_len = 30

In [0]:
tok = Tokenizer(char_level=False, split=' ')
tok.fit_on_texts(x_train)

In [15]:
vocab_len=len(tok.index_word.keys())
vocab_len

179838

In [16]:
seq_train = tok.texts_to_sequences(x_train)
matrix_train = sequence.pad_sequences(seq_train, maxlen=max_len)
matrix_train.shape

(914285, 30)

In [17]:
seq_test = tok.texts_to_sequences(x_test)
matrix_test = sequence.pad_sequences(seq_test, maxlen=max_len)
matrix_test.shape

(391837, 30)

In [18]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_len+1, 500, input_length=max_len))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(100 , activation='relu'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation="sigmoid"))
model_lstm.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

model_lstm.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 500)           89919500  
_________________________________________________________________
dropout (Dropout)            (None, 30, 500)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               240400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 90,160,001
Trainable params: 90,160,001
Non-trainable params: 0
_________________________________________________________________


In [28]:
model_lstm.fit(matrix_train,y_train.values , batch_size=7000 ,verbose=1, epochs = 1 , validation_data = (matrix_test,y_test.values))

131/131 [==============================] - 150s 1s/step - loss: 0.0374 - accuracy: 0.9850 - val_loss: 0.2946 - val_accuracy: 0.9462


In [0]:

model_lstm_json = model_lstm.to_json()
with open ("model_lstm_json.json","w")as json_file:
    json_file.write(model_lstm_json)
    
model_lstm.save_weights("model_lstm_json.h5")